In [5]:
import json
from collections import defaultdict

# Path to the combined annotations file
annotations_file = '/home/nele_pauline_suffo/ProcessedData/childlens_annotations/childlens_annotations.json'

def count_labels_and_durations(annotations_file):
    with open(annotations_file, 'r') as f:
        annotations = json.load(f)

    label_counts = defaultdict(int)
    label_durations = defaultdict(float)

    for video_id, data in annotations.items():
        for annotation in data['annotations']:
            label = annotation['label']
            label_counts[label] += 1
            segment = annotation['segment']
            duration = (segment[1] - segment[0])/60
            label_durations[label] += duration

    return label_counts, label_durations

label_counts, label_durations = count_labels_and_durations(annotations_file)

print("Label Counts:")
for label, count in label_counts.items():
    print(f'{label}: {count}')

print("\nLabel Durations (in minutes):")
for label, duration in label_durations.items():
    print(f'{label}: {duration:.2f}')

Label Counts:
Other Person Talking: 6113
Child Talking: 7447
Singing/Humming: 277
Drawing: 62
Overheard Speech: 1898
Pretend play: 59
Listening to Music/Audiobook: 68
Reading a Book: 81
Playing with Object: 317
Making Music: 3
Playing without Object: 25
Crafting Things: 26
Watching Something: 2
Dancing: 2

Label Durations (in minutes):
Other Person Talking: 455.29
Child Talking: 649.10
Singing/Humming: 82.00
Drawing: 374.91
Overheard Speech: 299.44
Pretend play: 158.84
Listening to Music/Audiobook: 222.14
Reading a Book: 328.70
Playing with Object: 1371.06
Making Music: 2.13
Playing without Object: 28.87
Crafting Things: 109.14
Watching Something: 5.09
Dancing: 0.57


In [6]:
import json
annotations = json.load(open("/home/nele_pauline_suffo/ProcessedData/childlens_annotations/childlens_annotations.json"))
annotations
annotations[key_dict[684742]]


NameError: name 'key_dict' is not defined

In [1]:
import cv2

def get_frame_count(video_path):
    # Open the vadeo file
    video = cv2.VideoCapture(video_path)
    
    # Check if the video was opened successfully
    if not video.isOpened():
        raise ValueError(f"Could not open the video file: {video_path}")
    
    # Get the frame count
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Release the video file
    video.release()
    
    return frame_count

# Example usage
video_path = "/home/nele_pauline_suffo/ProcessedData/videos_superannotate_all/805896.MP4"
print("Total number of frames:", get_frame_count(video_path))

#183537_Playing_with_Object_2 24310 0
#183537_Playing_with_Object_3 37752 0
#183537_Playing_with_Object_4 40367 0
#189224_Drawing_1 20973 2

#147984 18181
#204839 8946

#too long:
#207115 44104
#189224 53952
#183537 52719

Total number of frames: 33292


In [1]:
import torch
import pickle

def inspect_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    # Check if it's a PyTorch tensor
    if isinstance(data, torch.Tensor):
        print(f"Shape of tensor: {data.shape}")
        if len(data.shape) == 1:
            print("The tensor is 1D. It might represent flattened features.")
            print(f"Number of features: {data.shape[0]}")
        elif len(data.shape) == 2:
            print("The tensor is 2D.")
            print(f"Number of time steps (frames): {data.shape[0]}")
            print(f"Number of features per frame: {data.shape[1]}")
        else:
            print("Unexpected tensor shape. Inspect manually.")
    else:
        print(f"Unexpected data type: {type(data)}. Inspect manually.")
        print(data)


inspect_pickle('/home/nele_pauline_suffo/ProcessedData/videos_superannotate/features_flow/147984_Playing_with_Object_1.pkl')

/home/nele_pauline_suffo/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Shape of tensor: torch.Size([2048])
The tensor is 1D. It might represent flattened features.
Number of features: 2048


In [19]:
# load pkl file
with open('/home/nele_pauline_suffo/ProcessedData/videos_superannotate/features/147984_Playing_with_Object_1.pkl', 'rb') as f:
    data = pickle.load(f)
data

tensor([1.1846, 0.1259, 0.0621,  ..., 0.1813, 0.0496, 0.0961])